# TensorFlow Assignment: Convolutional Neural Network (CNN)

**[Duke Community Standard](http://integrity.duke.edu/standard.html): By typing your name below, you are certifying that you have adhered to the Duke Community Standard in completing this assignment.**

Name: 

### Convolutional Neural Network

Build a 2-layer CNN for MNIST digit classfication. Feel free to play around with the model architecture and see how the training time/performance changes, but to begin, try the following:

Image -> convolution (32 5x5 filters) -> nonlinearity (ReLU) ->  (2x2 max pool) -> convolution (64 5x5 filters) -> nonlinearity (ReLU) -> (2x2 max pool) -> fully connected (256 hidden units) -> nonlinearity (ReLU) -> fully connected (10 hidden units) -> softmax

Some tips:
- The CNN model might take a while to train. Depending on your machine, you might expect this to take up to half an hour. If you see your validation performance start to plateau, you can kill the training.

- Since CNNs a more complex than the logistic regression and MLP models you've worked with before, so you may find it helpful to use a more advanced optimizer. You're model will train faster if you use [`tf.train.AdamOptimizer`](https://www.tensorflow.org/api_docs/python/tf/train/AdamOptimizer) instead of `tf.train.GradientDescentOptimizer`. A learning rate of 1e-4 is a good starting point.

In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import trange     

/Users/musaidawan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# Import MNIST Data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
# 1st hidden layer with max pooling (convolution)

# Image Input 
# Input for MNIST data is flat. Reshap input image into a 4D batched image input
x_flat = tf.placeholder(tf.float32, [None, 784])
X0 = tf.reshape(x_flat, [-1, 28, 28, 1])

# Create convolutional kernel variable
W1 = tf.Variable(tf.truncated_normal([5, 5, 1, 32], stddev=0.1))

# Create bias variable
b1 = tf.Variable(tf.zeros([32]))

# Apply convolutional layer
conv1_preact = tf.nn.conv2d(X0, W1, strides=[1, 1, 1, 1], padding="SAME") + b1
conv1 = tf.nn.relu(conv1_preact)

# Print input/output shape
print(X0.shape)
print(conv1.shape)

# Max Pooling 
# Max pool and then print new shape
max_pool1 = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
print(max_pool1.shape)



(?, 28, 28, 1)
(?, 28, 28, 32)
(?, 14, 14, 32)


In [5]:
# 2nd hidden layer with max pooling (convolution)

# Create convolutional kernel variable
W2 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1))

# Create bias variable
b2 = tf.Variable(tf.zeros([64]))

# Apply convolutional layer
conv2_preact = tf.nn.conv2d(max_pool1, W2, strides=[1, 1, 1, 1], padding="SAME") + b2
conv2 = tf.nn.relu(conv2_preact)

# Print input/output shape
print(max_pool1.shape)
print(conv2.shape)

# Max Pooling 
# Max pool and then print new shape
max_pool2 = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
print(max_pool2.shape)
# 

(?, 14, 14, 32)
(?, 14, 14, 64)
(?, 7, 7, 64)


In [6]:
# 3rd hidden layer (fully connected with 256 hidden units + ReLu activation)

# Flatten convolutional feature maps into a vector
h_flat = tf.reshape(max_pool2, [-1, 7*7*64])

# Print output shape
print(h_flat.shape)

# Create weight matrix variable
W3 = tf.Variable(tf.truncated_normal([7*7*64, 256], stddev=0.1))
b3 = tf.Variable(tf.truncated_normal([256], stddev=0.1))

# Apply fully connected layer
A_preact_3 = tf.matmul(h_flat, W3) + b3
A3 = tf.nn.relu(A_preact_3)

print(A3.shape)

(?, 3136)
(?, 256)


In [7]:
# 4th hidden layer (fully connected with 10 hidden units + soft max activation)

# Create weight matrix variable
W4 = tf.Variable(tf.truncated_normal([256, 10], stddev=0.1))
b4 = tf.Variable(tf.truncated_normal([10], stddev=0.1))

# Apply fully connected layer
A_preact_4 = tf.matmul(A3, W4) + b4
A4 = tf.nn.softmax(A_preact_4)

print(A4.shape)
# This completes forward propagation where A4 gives the probability that the model thinks 
# that the image is of each of the 10 categories. Model predicts the max of these probabilities. 

(?, 10)


In [8]:
# Need to learn optimal W and b. 
    # Define Loss : Use cross entrophy, which is a common loss function for a classification problem 
    # Compute gradient using back propagation 

In [9]:
# Define Labels to evaluate the Loss Function 
y = tf.placeholder(tf.float32, [None, 10])

In [10]:
# Loss
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=A_preact_4, labels=y))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [11]:
# Optimizer
train_step = tf.train.GradientDescentOptimizer(0.05).minimize(cross_entropy)

In [12]:
# Session 
# To train, we simply call the optimizer op we defined above. 
# First though, we need to start a session and initialize our variables:

# Create a session object and initialize all graph variables
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [14]:
# Train the model
# trange is a tqdm function. It's the same as range, but adds a pretty progress bar
for epoch in trange(50):
    for which_batch in range(550):
        batch_xs = mnist.train.images[which_batch*100:(which_batch+1)*100]
        batch_ys = mnist.train.labels[which_batch*100:(which_batch+1)*100]
        #print(batch_xs.shape)
        #print(batch_ys.shape)
        sess.run(train_step, feed_dict={x_flat: batch_xs, y: batch_ys})
        
    # Test trained model 
    correct_prediction = tf.equal(tf.argmax(A_preact_4, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print('Test accuracy: {0}'.format(sess.run(accuracy, feed_dict={x_flat: mnist.test.images, y: mnist.test.labels})))
        


  2%|▏         | 1/50 [01:06<54:06, 66.26s/it]

Test accuracy: 0.978600025177002



  4%|▍         | 2/50 [02:14<53:42, 67.14s/it]

Test accuracy: 0.982699990272522



  6%|▌         | 3/50 [03:20<52:25, 66.93s/it]

...

Test accuracy: 0.9914000034332275



 98%|█████████▊| 49/50 [55:24<01:07, 67.85s/it]

Test accuracy: 0.9914000034332275



100%|██████████| 50/50 [56:31<00:00, 67.83s/it]


Test accuracy: 0.9914000034332275


In [ ]:
### YOUR CODE HERE ###




















### Short answer

1\. How does the CNN compare in accuracy with yesterday's logistic regression and MLP models? How about training time?

CNN is more accurate than Logistic regression and MLP. 

2\. How many trainable parameters are there in the CNN you built for this assignment?

*Note: By trainable parameters, I mean individual scalars. For example, a weight matrix that is 10x5 has 50.*

5*5*32 + 32 + 5*5*32*64 + 64 + 7*7*64*256 +  256 + 256*10 + 10 = 857738

3\. When would you use a CNN versus a logistic regression model or an MLP?

CNN is very useful for image data. Specially high resolution images with many channels. 